In [1]:
#imports here

import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os
import re
import pandas as pd

### This is where the fun begins

In [10]:
#create a function to change the date column into a datetime series
MONTH_TO_NUM = {"JAN" : "01", "FEV" : "02", "MAR" : "03", "AVR" : "04", "MAI" : "05", "JUN" : "06", "JUI" : "07", "AOU" : "08", "SEP" : "09", "OCT" : "10", "NOV" : "11", "DEC" : "12"}

def create_datetime(df, file_name):
    year= file_name[:4]

    #error handeling on invalid days anything greater than 31 (should add a clause to check per month type on year type as well):
    mask = pd.to_numeric(df["date"].str[:2]) <= 31
    df = df[mask]

    df["date"] = pd.to_datetime(year + df["date"].str[-3:].map(MONTH_TO_NUM) + df["date"].str[:2])
    if(file_name[5:7] == "01"):
        #mask used to 
        mask = df["date"].dt.month == 12
        df["date"][mask] -= pd.DateOffset(years = 1)
    return df

In [ ]:
#initalize all values here:
#capitals as per naming convention for python when you want to assign a constant
#padding needed to not read repetative noise
OFFSET = 6
#height in pixels of each row
HEIGHT = 27
#config for the ocr config arg
CONFIG = "--psm 12 --oem 1"
#collect the year for the files later by regular expression but for the time being, since we already know that the numbers are in the range of 0-9 position, we'll just slice

#set up paths
SOURCE_PATH = "data/unprocessed/debit/"
SAVE_PATH = "data/processed/debit/"
#-1 to remove the "/"
file_names = os.listdir(SOURCE_PATH)
for file in file_names:
    #create vars to hold each column (we will zip em together later to turn into a dataframe)
    #also reset values
    date = []
    description = []
    charge =[]
    total = []

    #here we will assume that the pdf size is 1700 to 2200 (later we will return to add more adaptability)
    #set up base crop dimensision
    left = 325
    top = 397
    right = 1650
    #bottom will be under constant change per loop, so we'll just initalize it
    bottom = top + HEIGHT

    #come back later to allow for more than one page to be processed
    image = convert_from_path(SOURCE_PATH + file)[1]

    #make shift do-while loop
    #we shall read all the way to the bottom, cleaning will be done in the next steps
    #we could switch it for a forloop sure by dividing the remainging length by the total rows height and just seeing how many times we can run it, but I already wrote this
    while bottom <= 2200:
        #Crop the image to just read row by row
        seg = image.crop((left, top, right, bottom))

        #Lets OCR this thing
        info = pytesseract.image_to_string(seg, config=CONFIG)
        print(info, file)
        info = info.rstrip("\n")
        info = info.split("\n\n")

        #some in precosion error handeling
        if len(info) > 3:
            date.append(info[0])
            description.append(info[1])
            charge.append(info[2])
            total.append(info[3])

        #reset the crop values
        top = bottom + OFFSET
        bottom = top + HEIGHT
    
    #turn into a df
    file_name = file[:10]
    df = pd.DataFrame(list(zip(date, description, charge, total)), columns=["date", "description", "charge", "total"])

    #here we clean the data in all the csvs

    #create a mask to filter out invalid data based on invalid dates (potentially remove and turn into a method to be used in other functions)
    mask = df["date"].str.contains(r"[0-3][0-9]\s*[A-Z]{3}")
    #remove exess
    df= df[mask]
    #To make sure we only have the formate we wish to use
    df['date'] = df["date"].str.extract(r"([0-3][0-9]\s*[A-Z]{3})", expand=False)

    #change the format of total and charge so we can use it without error later
    df["total"] = df["total"].replace(r"[^-*\d]", "", regex=True)
    df["charge"] = df["charge"].replace(r"[^\d]", "", regex=True)
    #turn it into a decimal
    df["total"] = df["total"].str[:-2] + "." + df["total"].str[-2:]
    df["charge"] = df["charge"].str[:-2] + "." + df["charge"].str[-2:]

    #add datetime:
    df = create_datetime(df,file)
    #index set to false as it is not needed
    df.to_csv(SAVE_PATH + file_name + ".csv", index=False)




To note over five years of data, only three anomolies showed up and one page where a second page was given to store more transaction information <br>
solution for anomoly: all three had the same 

In [14]:
#here we will put all the csvs into one big file to play around with
CSV_PATH = "data/processed/debit"
files = os.listdir(CSV_PATH)
#read all files as csvs and put into a list
files_to_concat = [pd.read_csv(CSV_PATH+'/'+file) for file in files]

main_df = pd.concat(files_to_concat, ignore_index=True)
expenses = main_df[main_df["total"].diff().fillna(0) <0]
main_df.to_csv("main.csv", index=False)
expenses.to_csv("expenses.csv", index=False)


,date,description,charge,total
1,2016-11-01,FRAIS FIXES,0.44,-2.94
2,2016-12-01,FR TENUE DE CPTE,2.50,-5.44
3,2016-12-01,FRAIS FIXES,2.75,-8.19
4,2016-12-01,INTERETS DECOUVERT,0.05,-8.24
6,2016-12-30,FRAIS FIXES,2.75,9.01
...,...,...,...,...
1387,2022-03-18,COMPTE DIVERS PAYPAL,24.58,1828.76
1388,2022-03-22,VIREMENT INTERAC,96.14,1732.62
1390,2022-03-28,MOBILE VIREMENTMC,728.92,1103.70
1391,2022-03-29,COMPTE DIVERS PAYPAL,44.43,1059.27


In [6]:
main_df['total']

0        2.50
1        2.94
0        5.44
1        8.19
2        8.24
       ...   
11    1832.62
12    1103.70
13    1059.27
14    2035.94
15     535.94
Name: total, Length: 1396, dtype: float64

Due to time restraints I had to fix some files by hand so that the values I'd use for the model would be at their most accurate

In [ ]:
#here we merge all csv's into one big